In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import io
from PIL import Image
import numpy as np
import tensorflow as tf

app = FastAPI()

# Allow frontend requests (Flutter)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Change to Flutter app domain in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the trained model
model = tf.keras.models.load_model("C:/xampp/htdocs/php-practics/fnapp/assets/images/food_classifier_model.h5")  # Adjust model path

# Preprocessing function
def preprocess_image(image: Image.Image):
    image = image.resize((224, 224))  # Resize to model input size
    img_array = np.array(image) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for model
    return img_array

@app.post("/predict")
async def predict_food(image: UploadFile = File(...)):
    contents = await image.read()
    img = Image.open(io.BytesIO(contents)).convert("RGB")
    
    processed_img = preprocess_image(img)
    predictions = model.predict(processed_img)
    
    class_labels = ["Pizza", "Burger", "Salad", "Pasta", "Sushi"]  # Adjust with your model classes
    predicted_label = class_labels[np.argmax(predictions)]

    return {"food_name": predicted_label}
